# SDC and Terminologies 

The basic building block of FDF files is the Question-Answer Set (QAS). The QAS model follows the common practice of representing information as Question-answer pairs. In the eCP world, this translates into storing a Question @ID and a ListItem @ID for each Question and ListItem in the filled-out DEF. Although it is possible to use SNOMED CT, LOINC or other semantic coding systems for this purpose, complete sets of these codes for each FDF are not always available or stable over time. Therefore, the @ID is a convenient surrogate key for the Questions and ListItems. @IDs are mapped, in separate XML mapping files, to the appropriate semantic codes, depending on code availability and end-user needs. Thus, @IDs provide unique identifiers for lines in a template that can map to external coding systems such as LOINC, SNOMED CT, and NAACCR data items.

To enable querying based on external coding systems, code tables for these systems should be stored separately from the @ID-based QAS data. External codes should be linked to the Question/ListItem pairs by using the @ID as a “foreign key” in a mapping table for the external codes. The eCP team provides these @ID/external code mapping tables in XML format, so there is no need to maintain them on your own.

Since many coding systems (e.g., SNOMED CT and ICD-O-3) introduce changes to the code values or the meaning of codes, storing external codes together with the @IDs would require regular data maintenance, would risk the introduction of errors, and might result in the editing of an electronically-signed and locked patient record. In addition, external codes may not be available at the time of data storage. By using a mapping table, external codes can be simply referenced via the @ID; updates to the mapping table can be as simple as importing an updated eCP mapping file.

For some applications, it may be necessary to transmit semantic (e.g., SNOMED CT, ICD-O-3), terminological (e.g., LOINC) or use-case-specific (e.g., NAACCR codes) codes in addition to @IDs. Creating HL7 messages that include non-@ID coding systems is straightforward when using the provided eCP mappings between @IDs and external coding systems. In Canada, the exact types and locations of the codes in HL7 messages are subject to standardization by the North American Association of Cancer Registries (NAACCR), and also by governmental agencies such as Canada Health Infoway (CHI), Canadian Partnership against Cancer (CPAC) and Cancer Care Ontario (CCO).

## Context in SDC and Terminologies 

Context refers to the hierarchical nesting relationship between form components defined by the FDF XML, and the effect of this relationship on the meaning and interpretation of the nested DE components. Context (especially ancestors) can greatly affect the semantics of the DE units in an FDF and DEF.

Incorrectly separating the DEF user responses into separate data slots (e.g., “shredding” captured QAS responses into database tables and fields not designed for SDC) can result in loss of data integrity and could lead to erroneous conclusions. Care must be taken to assure the preservation of the original context.

Context is especially important to consider when storing DEF responses and assigning terminologies/codes. Preservation of context is a major advantage of SDC over other technologies. Terminologies (e.g., SNOMED CT codes) are not a reliable substitute for SDC context preservation.

## Terminology (ICD-O-3 & SNOMED CT) Maps
CCPs and eCP FDF files are released before the ICD-O-3 and SNOMED CT encodings are available. In general, limited @ID to terminology maps will be provided 6-12 months after FDF files are released.

To provide a functional and interoperable implementation of the CAP eCP, it is critical that neither SNOMED CT codes (i.e., Concept IDs), nor codes from any other terminology, are used as unique identifiers for CAP eCP Questions and ListItems. @ID values should be stored for each Question and for each selected ListItem in the template.

Currently mapping tables are distributed as Microsoft Excel Spreadsheets, however that will evolve to be SDC XML based mapping files in the future.

## Data Storage
When storing the ListItem data for a Single-Select Question, a single Question code (the “@ID” of the Question item) and a single ListItem code (the “@ID” of the ListItem) are stored in the database.

When storing ListItem data for a Multi-Select Question, a single Question code (the @ID of the Question item) and a single ListItem code (the @ID of the ListItem) are stored in a database record for each selected ListItem (checked box).

Why is it important to store the Question @ID in addition to the ListItem @ID? During eCP updates, sometimes Questions are deprecated to indicate a change to the Question’s value (ListItem) list, but some or all of the original ListItem remain in the list (they are not deprecated). Replacement of the Question (and its @ID) ensures that new queries based on the previous Question @ID will not return ListItems from the new Question. This is intended to prevent misinterpretation of query data when values to the value list (ListItems) change. In other words, queries based on the deprecated Question will not return combined results from the new and the deprecated Questions, since these Questions are based on different ListItem sets.

If the Question has not been answered, then neither the Question @ID nor the ListItem @ID need be stored. In the case of LIR items, the text of the ListItem’s Response section should be stored in addition to the ListItem’s @ID. Following this model will help to ensure that the stored data is interoperable with other template implementations.

## Embedding Terminology in Forms

Terminologies may also be mapped directly within SDC forms using the `CodedValue` element. The `CodedValue` element has several sub elements which are included when it is used

* `Code`: the actual value of code from the specified terminology
* `CodeText`: the descriptive text of the code
* `CodeSystem`: contains information about the code system in sub elements
    * `CodeSystemName`: the name of the code system

`CodedValue`is described in more value in the SDC Schema section of this Polyglot notebook.

It is strongly recommended that the terminology be stored in a separate mapping file as described above. Placing the terminology directly in the form can have benefits, such as avoiding a potential loss of context, however it is often not worth the increased message size. 

There are examples, such as Clinical Decision Support use cases, where the code must be mapped inside the form to trigger some kind of other action. These use cases are rare and will be elaborated on in the SDC OM Section of this Polyglot notebook.

The example below shows a form where the terminology has been directly embedded using the `CodedValue` element.

In [ ]:
					<Section title="CLINICAL INFORMATION" type="level1" ID="000001">
						<CodedValue>
							<Code val="55752-0" />
							<CodeText val="Clinical Information" />
							<CodeSystem>
								<CodeSystemName val="LOINC" />
							</CodeSystem>
						</CodedValue>
						<ChildItems>
							<Question title="Clinical Information:" ID="000002">
								<Property propClass="numbering" propName="numbering" val="1." />
								<ResponseField>
									<Response>
										<string val="" />
									</Response>
								</ResponseField>
							</Question>
							<Question title="Reason for exam:" ID="000004">
								<Property propClass="numbering" propName="numbering" val="2." />
								<CodedValue>
									<Code val="RID10393" />
									<CodeText val="imaging purpose" />
									<CodeSystem>
										<CodeSystemName val="RADLEX" />
									</CodeSystem>
								</CodedValue>
								<ListField>
									<List>
										<ListItem title="Baseline scan" ID="000005">
											<CodedValue>
												<Code val="RID49571" />
												<CodeText val="Baseline" />
												<CodeSystem>
													<CodeSystemName val="RADLEX" />
												</CodeSystem>
											</CodedValue>
										</ListItem>
									</List>
								</ListField>
							</Question>
						</ChildItems>
					</Section>

## Mapping Terminology using FHIR

a FHIR ConceptMap may be used to map terminology to codes within in any SDC form including eCPs. 